# FX Rate Conversion Agent with External API Integration

This notebook demonstrates **external API integration** for agent workflows by building a currency conversion agent that:

1. **Fetches live FX rates** from a free public API (Frankfurter)
2. **Calculates total costs** including FX fees and sales tax
3. **Provides itemized breakdowns** with clear explanations
4. **Handles missing inputs** with clarifying questions

## Key Concepts Demonstrated

- **External API Integration**: Live data fetching from public APIs
- **Agent Tool Development**: Custom tools for specific workflows
- **Error Handling**: Graceful API failure management
- **Input Validation**: Missing parameter detection and clarification
- **Financial Calculations**: Multi-step fee and tax computations

## Scenario
A shopping assistant that helps users understand the true cost of international purchases by converting prices to USD with realistic fees and taxes.

In [1]:
import os
import json
import requests
from datetime import datetime
from typing import Dict, Optional, Tuple
from dataclasses import dataclass
from openai import OpenAI

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)


In [2]:
@dataclass
class PurchaseRequest:
    """Data model for purchase conversion request"""
    price: Optional[float] = None
    from_ccy: Optional[str] = None
    to_ccy: str = "USD"
    fx_fee_pct: Optional[float] = None
    sales_tax_pct: Optional[float] = None
    
    def get_missing_fields(self) -> list:
        """Return list of required fields that are missing"""
        required_fields = []
        if self.price is None:
            required_fields.append("price")
        if self.from_ccy is None:
            required_fields.append("from_ccy")
        if self.fx_fee_pct is None:
            required_fields.append("fx_fee_pct")
        if self.sales_tax_pct is None:
            required_fields.append("sales_tax_pct")
        return required_fields
    
    def is_complete(self) -> bool:
        """Check if all required fields are provided"""
        return len(self.get_missing_fields()) == 0

@dataclass 
class ConversionResult:
    """Data model for conversion calculation results"""
    spot_rate: float
    base_amount_usd: float
    fx_fee_usd: float
    sales_tax_usd: float
    total_usd: float
    from_ccy: str
    to_ccy: str
    fx_fee_pct: float
    sales_tax_pct: float

In [3]:
class FXRateAgent:
    """Currency conversion agent with external API integration"""
    
    def __init__(self):
        self.fx_api_base = "https://api.frankfurter.app"
        
    def get_fx_rate(self, from_ccy: str, to_ccy: str) -> Optional[float]:
        """
        Fetch live FX rate from Frankfurter API
        
        Args:
            from_ccy: Source currency code (e.g., 'EUR', 'GBP')
            to_ccy: Target currency code (e.g., 'USD')
            
        Returns:
            Exchange rate as float, or None if API call fails
        """
        try:
            # Frankfurter API endpoint for latest rates
            url = f"{self.fx_api_base}/latest"
            params = {
                "from": from_ccy.upper(),
                "to": to_ccy.upper()
            }
            
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            
            data = response.json()
            
            # Extract the exchange rate
            if to_ccy.upper() in data.get("rates", {}):
                return data["rates"][to_ccy.upper()]
            else:
                print(f"Rate not found for {from_ccy} to {to_ccy}")
                return None
                
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return None
        except (KeyError, ValueError) as e:
            print(f"Error parsing API response: {e}")
            return None
    
    def calc_total(self, price: float, from_ccy: str, to_ccy: str, 
                   fx_rate: float, fx_fee_pct: float, sales_tax_pct: float) -> ConversionResult:
        """
        Calculate total cost including FX fees and sales tax
        
        Args:
            price: Original price in source currency
            from_ccy: Source currency code
            to_ccy: Target currency code  
            fx_rate: Exchange rate (from -> to)
            fx_fee_pct: FX fee percentage (e.g., 3.0 for 3%)
            sales_tax_pct: Sales tax percentage (e.g., 6.0 for 6%)
            
        Returns:
            ConversionResult with detailed breakdown
        """
        # Step 1: Convert base amount
        base_amount_usd = price * fx_rate
        
        # Step 2: Calculate FX fee on the converted amount
        fx_fee_usd = base_amount_usd * (fx_fee_pct / 100)
        
        # Step 3: Calculate sales tax on base amount (before FX fee)
        sales_tax_usd = base_amount_usd * (sales_tax_pct / 100)
        
        # Step 4: Calculate total
        total_usd = base_amount_usd + fx_fee_usd + sales_tax_usd
        
        return ConversionResult(
            spot_rate=fx_rate,
            base_amount_usd=base_amount_usd,
            fx_fee_usd=fx_fee_usd,
            sales_tax_usd=sales_tax_usd,
            total_usd=total_usd,
            from_ccy=from_ccy,
            to_ccy=to_ccy,
            fx_fee_pct=fx_fee_pct,
            sales_tax_pct=sales_tax_pct
        )
    
    def extract_purchase_info(self, user_input: str) -> PurchaseRequest:
        """
        Extract purchase information from user input using LLM
        
        Args:
            user_input: Natural language purchase request
            
        Returns:
            PurchaseRequest with extracted information
        """
        extraction_prompt = f"""
Extract purchase information from this user input: "{user_input}"

Extract and return ONLY the information in this exact JSON format:
{{
    "price": number or null,
    "from_ccy": "currency code or null",
    "to_ccy": "currency code or USD",
    "fx_fee_pct": number or null,
    "sales_tax_pct": number or null
}}

Rules:
- For price: extract numeric value only (e.g., "€1,299" → 1299)
- For currencies: use 3-letter codes (EUR, GBP, USD, etc.)
- For percentages: extract numeric value (e.g., "3%" → 3)
- Use null for missing information
- Default to_ccy is "USD" if not specified
- Look for keywords: FX fee, foreign exchange fee, sales tax, tax rate

Return valid JSON only, no explanations.
"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.1,
                max_tokens=200
            )
            
            response_content = response.choices[0].message.content.strip()
            
            # Clean up response to extract JSON
            if "```json" in response_content:
                response_content = response_content.split("```json")[1].split("```")[0].strip()
            elif "```" in response_content:
                response_content = response_content.split("```")[1].strip()
            
            data = json.loads(response_content)
            
            return PurchaseRequest(
                price=data.get("price"),
                from_ccy=data.get("from_ccy"),
                to_ccy=data.get("to_ccy", "USD"),
                fx_fee_pct=data.get("fx_fee_pct"),
                sales_tax_pct=data.get("sales_tax_pct")
            )
            
        except Exception as e:
            print(f"Error extracting purchase info: {e}")
            return PurchaseRequest()
    
    def ask_clarifying_question(self, missing_fields: list) -> str:
        """
        Generate one clarifying question for missing information
        
        Args:
            missing_fields: List of missing required fields
            
        Returns:
            Natural language clarifying question
        """
        if not missing_fields:
            return ""
            
        # Prioritize the most important missing field
        priority_order = ["price", "from_ccy", "fx_fee_pct", "sales_tax_pct"]
        
        for field in priority_order:
            if field in missing_fields:
                questions = {
                    "price": "What's the price of the item you want to buy?",
                    "from_ccy": "What currency is the price in (e.g., EUR, GBP)?",
                    "fx_fee_pct": "What's your card's foreign exchange fee percentage?",
                    "sales_tax_pct": "What's the sales tax rate in your location (use 0 if no tax)?"
                }
                return questions[field]
        
        return "Could you provide more details about your purchase?"
    
    def format_result(self, result: ConversionResult) -> str:
        """
        Format conversion result into user-friendly breakdown
        
        Args:
            result: ConversionResult object with calculation details
            
        Returns:
            Formatted string with itemized breakdown
        """
        return f"""💰 **Purchase Conversion Breakdown**

**Spot Rate**: 1 {result.from_ccy} = {result.spot_rate:.4f} {result.to_ccy}
**Base Amount**: ${result.base_amount_usd:.2f} USD
**FX Fee ({result.fx_fee_pct}%)**: ${result.fx_fee_usd:.2f} USD
**Sales Tax ({result.sales_tax_pct}%)**: ${result.sales_tax_usd:.2f} USD

**📊 Grand Total**: ${result.total_usd:.2f} USD

*Exchange rate from Frankfurter API as of {datetime.now().strftime('%Y-%m-%d %H:%M')} UTC*"""
    
    def process_purchase_request(self, user_input: str) -> str:
        """
        Main processing function for purchase conversion requests
        
        Args:
            user_input: Natural language purchase request
            
        Returns:
            Either clarifying question or complete conversion breakdown
        """
        # Extract purchase information
        request = self.extract_purchase_info(user_input)
        
        # Check for missing information
        missing_fields = request.get_missing_fields()
        if missing_fields:
            return self.ask_clarifying_question(missing_fields)
        
        # Fetch live FX rate
        fx_rate = self.get_fx_rate(request.from_ccy, request.to_ccy)
        if fx_rate is None:
            return f"❌ Unable to fetch current exchange rate for {request.from_ccy} to {request.to_ccy}. Please check your internet connection and try again."
        
        # Calculate total with fees
        result = self.calc_total(
            price=request.price,
            from_ccy=request.from_ccy,
            to_ccy=request.to_ccy,
            fx_rate=fx_rate,
            fx_fee_pct=request.fx_fee_pct,
            sales_tax_pct=request.sales_tax_pct
        )
        
        # Return formatted breakdown
        return self.format_result(result)

## Demo: Interactive FX Rate Conversion

Let's test the agent with various purchase scenarios to demonstrate:
1. **Live API integration** with real exchange rates
2. **Missing information handling** with clarifying questions
3. **Complete calculations** with itemized breakdowns
4. **Error handling** for API failures

In [4]:
# Initialize the FX Rate Agent
agent = FXRateAgent()

def test_scenarios():
    """Test the agent with various purchase scenarios"""
    
    test_cases = [
        "If a laptop costs €1,299 and my card has a 3% FX fee, what's the total in USD today (assume 6% sales tax in my state)?",
        "Price is £85, no sales tax, 2% FX fee. How much in USD?",
        "€49.99, tax unknown, FX fee 3% → ask one clarifying Q, then compute.",
        "I want to buy something for ¥15000 with 2.5% FX fee and 8.5% sales tax",
        "The item costs 500 CHF, what do I need to tell you?"
    ]
    
    print("🌍 FX RATE CONVERSION AGENT DEMO")
    print("=" * 60)
    print()
    
    for i, test_input in enumerate(test_cases, 1):
        print(f"📝 **Test Case {i}:**")
        print(f"User: {test_input}")
        print()
        
        # Process the request
        response = agent.process_purchase_request(test_input)
        print(f"🤖 Agent: {response}")
        print()
        print("-" * 50)
        print()

# Run the test scenarios
test_scenarios()

🌍 FX RATE CONVERSION AGENT DEMO

📝 **Test Case 1:**
User: If a laptop costs €1,299 and my card has a 3% FX fee, what's the total in USD today (assume 6% sales tax in my state)?

🤖 Agent: 💰 **Purchase Conversion Breakdown**

**Spot Rate**: 1 EUR = 1.1649 USD
**Base Amount**: $1513.21 USD
**FX Fee (3%)**: $45.40 USD
**Sales Tax (6%)**: $90.79 USD

**📊 Grand Total**: $1649.39 USD

*Exchange rate from Frankfurter API as of 2025-10-17 00:11 UTC*

--------------------------------------------------

📝 **Test Case 2:**
User: Price is £85, no sales tax, 2% FX fee. How much in USD?

🤖 Agent: What's the sales tax rate in your location (use 0 if no tax)?

--------------------------------------------------

📝 **Test Case 3:**
User: €49.99, tax unknown, FX fee 3% → ask one clarifying Q, then compute.

🤖 Agent: What's the sales tax rate in your location (use 0 if no tax)?

--------------------------------------------------

📝 **Test Case 4:**
User: I want to buy something for ¥15000 with 2.5% FX fee 

## Individual Tool Testing

Let's test each tool individually to understand how they work:

In [5]:
print("🔧 INDIVIDUAL TOOL TESTING")
print("=" * 40)
print()

# Test 1: FX Rate API Integration
print("1️⃣ **Testing get_fx_rate() tool:**")
test_rates = [
    ("EUR", "USD"),
    ("GBP", "USD"), 
    ("JPY", "USD"),
    ("CHF", "USD")
]

for from_ccy, to_ccy in test_rates:
    rate = agent.get_fx_rate(from_ccy, to_ccy)
    if rate:
        print(f"   {from_ccy} → {to_ccy}: {rate:.4f}")
    else:
        print(f"   {from_ccy} → {to_ccy}: Failed to fetch rate")

print()

# Test 2: Calculation Tool
print("2️⃣ **Testing calc_total() tool:**")
print("   Scenario: €1,299 laptop with 3% FX fee and 6% sales tax")

# Use a recent EUR/USD rate (or fetch it live)
eur_usd_rate = agent.get_fx_rate("EUR", "USD")
if eur_usd_rate:
    result = agent.calc_total(
        price=1299,
        from_ccy="EUR",
        to_ccy="USD", 
        fx_rate=eur_usd_rate,
        fx_fee_pct=3.0,
        sales_tax_pct=6.0
    )
    
    print(f"   Original Price: €1,299")
    print(f"   Exchange Rate: {result.spot_rate:.4f}")
    print(f"   Base USD: ${result.base_amount_usd:.2f}")
    print(f"   FX Fee: ${result.fx_fee_usd:.2f}")
    print(f"   Sales Tax: ${result.sales_tax_usd:.2f}")
    print(f"   Total: ${result.total_usd:.2f}")
else:
    print("   Could not fetch exchange rate for calculation test")

print()

# Test 3: Information Extraction
print("3️⃣ **Testing extract_purchase_info() tool:**")
test_extractions = [
    "€1,299 laptop with 3% FX fee and 6% sales tax",
    "£85 item, 2% FX fee, no tax",
    "¥15000 purchase, missing fee info"
]

for text in test_extractions:
    request = agent.extract_purchase_info(text)
    print(f"   Input: '{text}'")
    print(f"   Extracted: price={request.price}, from_ccy={request.from_ccy}, fx_fee={request.fx_fee_pct}%, tax={request.sales_tax_pct}%")
    print()

🔧 INDIVIDUAL TOOL TESTING

1️⃣ **Testing get_fx_rate() tool:**
   EUR → USD: 1.1649
   GBP → USD: 1.3427
   JPY → USD: 0.0066
   CHF → USD: 1.2523

2️⃣ **Testing calc_total() tool:**
   Scenario: €1,299 laptop with 3% FX fee and 6% sales tax
   Original Price: €1,299
   Exchange Rate: 1.1649
   Base USD: $1513.21
   FX Fee: $45.40
   Sales Tax: $90.79
   Total: $1649.39

3️⃣ **Testing extract_purchase_info() tool:**
   Input: '€1,299 laptop with 3% FX fee and 6% sales tax'
   Extracted: price=1299, from_ccy=EUR, fx_fee=3%, tax=6%

   Input: '£85 item, 2% FX fee, no tax'
   Extracted: price=85, from_ccy=GBP, fx_fee=2%, tax=None%

   Input: '¥15000 purchase, missing fee info'
   Extracted: price=15000, from_ccy=JPY, fx_fee=None%, tax=None%



## Error Handling and Edge Cases

Let's test how the agent handles various error conditions and edge cases:

In [6]:
print("⚠️ ERROR HANDLING & EDGE CASES")
print("=" * 40)
print()

# Test API Error Handling
print("1️⃣ **Testing invalid currency codes:**")
invalid_rate = agent.get_fx_rate("INVALID", "USD")
print(f"   Invalid currency result: {invalid_rate}")
print()

# Test Missing Information Handling
print("2️⃣ **Testing missing information scenarios:**")
incomplete_requests = [
    "I want to buy something expensive",
    "€500 item",
    "Something costs £200 with 3% FX fee"
]

for req in incomplete_requests:
    print(f"   Input: '{req}'")
    response = agent.process_purchase_request(req)
    print(f"   Response: {response}")
    print()

# Test Edge Cases
print("3️⃣ **Testing edge cases:**")

# Zero fees
print("   Zero fees scenario:")
gbp_usd_rate = agent.get_fx_rate("GBP", "USD")
if gbp_usd_rate:
    zero_fee_result = agent.calc_total(
        price=100,
        from_ccy="GBP",
        to_ccy="USD",
        fx_rate=gbp_usd_rate,
        fx_fee_pct=0.0,
        sales_tax_pct=0.0
    )
    print(f"   £100 with 0% fees = ${zero_fee_result.total_usd:.2f} USD")

# High fees
print("   High fees scenario:")
if gbp_usd_rate:
    high_fee_result = agent.calc_total(
        price=100,
        from_ccy="GBP", 
        to_ccy="USD",
        fx_rate=gbp_usd_rate,
        fx_fee_pct=10.0,
        sales_tax_pct=15.0
    )
    print(f"   £100 with 10% FX fee + 15% tax = ${high_fee_result.total_usd:.2f} USD")

⚠️ ERROR HANDLING & EDGE CASES

1️⃣ **Testing invalid currency codes:**
API request failed: 404 Client Error: Not Found for url: https://api.frankfurter.app/latest?from=INVALID&to=USD
   Invalid currency result: None

2️⃣ **Testing missing information scenarios:**
   Input: 'I want to buy something expensive'
   Response: What's the price of the item you want to buy?

   Input: '€500 item'
   Response: What's your card's foreign exchange fee percentage?

   Input: 'Something costs £200 with 3% FX fee'
   Response: What's the sales tax rate in your location (use 0 if no tax)?

3️⃣ **Testing edge cases:**
   Zero fees scenario:
   £100 with 0% fees = $134.27 USD
   High fees scenario:
   £100 with 10% FX fee + 15% tax = $167.84 USD


## Key Learning Points Demonstrated

### 1. **External API Integration**
- **Live Data Fetching**: Real-time exchange rates from Frankfurter API
- **Error Handling**: Graceful degradation when APIs are unavailable
- **API Response Processing**: JSON parsing and data extraction
- **Timeout Management**: Robust request handling with timeouts

### 2. **Agent Tool Development**
- **Tool Composition**: Multiple specialized tools working together
- **Tool Orchestration**: Coordinated workflow across API calls and calculations
- **Tool Error Recovery**: Individual tool failure handling without breaking the workflow

### 3. **Input Processing and Validation**
- **Natural Language Extraction**: LLM-powered information extraction from user input
- **Missing Data Handling**: Intelligent clarifying questions for incomplete requests
- **Data Validation**: Type checking and format validation for extracted information

### 4. **Financial Calculations**
- **Multi-step Computations**: Base conversion, fee calculation, tax application
- **Transparent Breakdowns**: Clear itemization of all cost components
- **Accurate Arithmetic**: Proper order of operations for financial calculations

### 5. **Production Considerations**
- **API Rate Limiting**: Considerations for high-volume usage
- **Caching Strategies**: Potential for rate caching to reduce API calls
- **Fallback Mechanisms**: Alternative data sources when primary APIs fail
- **Currency Support**: Extensible framework for additional currencies
- **Audit Trails**: Timestamped results for financial record keeping

### 6. **User Experience Design**
- **Progressive Disclosure**: Ask for one missing piece of information at a time
- **Clear Communication**: Friendly, formatted output with all necessary details
- **Error Transparency**: Clear error messages when things go wrong
- **Professional Output**: Suitable for financial decision making

This pattern can be extended to other external APIs like:
- **Stock prices** for investment agents
- **Weather data** for travel planning agents  
- **Product catalogs** for shopping agents
- **Real estate data** for property search agents